In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline

from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

# import os

1. dailyhourlyMean()함수로 요일별, 시간별 평균 계산한 df_mean생성
2. for loop으로 df 칼럼별
<br>- dacon방식으로 직후 Null이 1개 이상 존재하는 인덱스 추출, 
<br>- df_mean에서 해당 Null시간에 따른 시간 평균 추출,
<br>- 시간별 가중치는 (해당시간 평균/null인 시간들의 평균 합)으로 계산
<br>- 누적치/null개수 * 가중치 
<br> : threshold보다 클 경우 그대로 입력, 그렇지 않을 경우 시간평균으로 대치

In [2]:
test = pd.read_csv("data/test.csv", sep = ",")
train = pd.read_csv("data/train.csv", sep =",")
# submission = pd.read_csv("data/submission.csv")

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [3]:
# train = pd.read_csv("/kaggle/input/traindata/train.csv", sep=',')
# test = pd.read_csv("/kaggle/input/testdata/test.csv", sep=',')

In [4]:
def dailyhourlyMean(df):
    df['Time'] = pd.to_datetime(df.Time) ; df = df.set_index('Time')
    df['hour'] = df.index.hour ;  df['day'] = df.index.dayofweek

    return df.columns[:-2], df.groupby(['day','hour']).mean()

In [5]:
def DaconDHmeanPrep(df): #이 때 df는 아직 time Index가 안 되어있는 raw
    ids, df_mean = dailyhourlyMean(df)
    df1 = df
    for k in range(1,len(df.columns)):
        counting = df.loc[ df.iloc[:,k].isnull()==False ][ df.columns[k] ].index
        _id = pd.DataFrame(list(zip(counting[:-1], #값이 존재하는 index(시간) 중 마지막 제외
                          counting[1:]-counting[:-1] - 1)), # index 중 첫번째 제외 - 마지막 제외 - 1 : 각 인덱스 사이 구간에 얼마의 차이가 있는가, 즉 Null값이 몇개나 존재했는가
                 columns=['index','count'] )
        p005 = np.percentile(df.iloc[:,k].dropna(), 0.5)
        med = df.iloc[:,k].median()

        na_ids = _id[(_id['count'] >0)].reset_index(drop=True)
        if k%50 ==0: 
            print(k)
        for i, j in zip(na_ids['index'], na_ids['count']):
            initial = df.iloc[i,k]
            if initial > med:
                timeid = pd.to_datetime(df.iloc[i,0])
                timerange = timeid + pd.to_timedelta (np.arange(j+1), 'h')
                dh = [(t.dayofweek, t.hour) for t in timerange]
                means = df_mean.loc[dh, ids[k-1]]
                mean_tot = means.sum()
                weights = means/mean_tot
                w_values = weights*df.iloc[i,k]
                wv = pd.DataFrame(list(map(lambda x:x if x>p005 else np.nan, 
                                      df.iloc[i,k]*weights.values)))
                if np.isnan(wv.iloc[0,0]):
                    wv.iloc[0,0] = initial
                df1.iloc[i:i+j+1, k] = wv.values   
                # print('column ', ids[k-1], '| row', i, '| na count ',j,'\ntimerange ',timerange,'\nweights',weights, wv)
    df1['Time'] = pd.to_datetime(df1.Time) ; df1 = df1.set_index('Time') 
    return df1      

In [ ]:
train2 = train.dropna(how='all', axis=1).copy()
changed_train = DaconDHmeanPrep(train2)
changed_train.head()

In [ ]:
test2 = test.dropna(how='all', axis=1).copy()
changed_test = DaconDHmeanPrep(test2)
changed_test.head()

In [ ]:
changed_test.to_csv("test_DHWeight_onDacon.csv")
# from IPython.display import FileLink
# FileLink(r'test_DHWeight_onDacon.csv')

In [ ]:
changed_train.to_csv("train_DHWeight_onDacon.csv")
# from IPython.display import FileLink
# FileLink(r'train_DHWeight_onDacon.csv')

In [ ]:
_, ax = plt.subplots(2,2, figsize=(20,15)) 

a=sns.distplot(test.isnull().mean(axis=0), ax=ax[0,0]) #나열된 값을 distplot을 이용해 시각화 하고, 이를 첫 번째 그래프 창에 넣습니다.
a.set(ylim=(0, 30))
ax[0,0].set_title('Distribution of Missing Values Percentage in Test set')

b=sns.distplot(changed_test.isnull().mean(axis=0), ax=ax[0,1]) #test data에서의 결측치 비율을 시각화 하고, 이를 두 번째 그래프 창에 넣습니다.
b.set(ylim=(0, 30))
ax[0,1].set_title('Distribution of Missing Values Percentage in Test set after Missing value processing')

a=sns.distplot(train.isnull().mean(axis=0), ax=ax[1,0]) #나열된 값을 distplot을 이용해 시각화 하고, 이를 첫 번째 그래프 창에 넣습니다.
a.set(ylim=(0, 30))
ax[1,0].set_title('Distribution of Missing Values Percentage in TRAIN set')

b=sns.distplot(changed_train.isnull().mean(axis=0), ax=ax[1,1]) #test data에서의 결측치 비율을 시각화 하고, 이를 두 번째 그래프 창에 넣습니다.
b.set(ylim=(0, 30))
ax[1,1].set_title('Distribution of Missing Values Percentage in TRAIN set after Missing value processing')
plt.show()

### Arima

In [4]:
# !pip install pmdarima

In [23]:
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
import pmdarima as pm
from pmdarima.arima import auto_arima
# from pmdarima.datasets import load_wineind
import warnings
warnings.filterwarnings('ignore')

In [7]:
changed_train = pd.read_csv("data/train_DHWeight_onDacon.csv", sep=',')
changed_test = pd.read_csv("data/test_DHWeight_onDacon.csv", sep=',')

In [8]:
changed_test['Time'] = pd.to_datetime(changed_test['Time']) 
changed_test = changed_test.set_index('Time')

In [9]:
idx = changed_test.count(axis = 'columns') >= 201*0.5
ch_te_TimeCut = changed_test[idx]

In [32]:
changed_train['Time'] = pd.to_datetime(changed_train['Time']) 
changed_train = changed_train.set_index('Time')

In [33]:
idx = changed_train.count(axis = 'columns') >= 1292*0.5
ch_tr_TimeCut = changed_train[idx]

In [11]:
def dfPrep(df):
    time = []; place_id = []; target = []  
#     df['Time'] = pd.to_datetime(df['Time']) 
#     df = df.set_index('Time')
    for i in df.columns:
        for j in range(len(df)):
            place_id.append(i)
            time.append(df.index[j])
            target.append(df[i].iloc[j])
    df2 = pd.DataFrame({'place_id': place_id, 'time':time, 'target': target})
    df2 = df2.dropna().set_index('time')
    return df2

In [12]:
def get_optimal_params(y):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    param_dict = {}
    for param in pdq:
        try:
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            param_dict[results_ARIMA.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

In [30]:
def runArima(df):
    df2 = dfPrep(df)
    agg={}
    n = 0
    hrs = [str(i+1) for i in range(24)] 
    for key in df2['place_id'].unique(): # 미터ID리스트를 unique()함수로 추출, for loop
        n+=1   
        temp = df2.loc[df2['place_id']==key] # new_test2에서 key와 일치하는 place_id를 가지는 부분을 temp에 할당
       # 1 시간별 예측
        model = ARIMA(temp['target'],  # target 을 추측하고자 함
                      order=get_optimal_params(temp['target'])) # AIC를 최소화하는 최적의 파라미터 
#         model = auto_arima(temp['target'].values, 
#                   error_action='ignore',supress_warnings=True,
#                   seasonal=True, m=12)
        resultsArima = model.fit(disp=-1)
        print(n, key, 'predicting')
        fcst1 = resultsArima.forecast(24) # 24시간 예측
        a = pd.DataFrame() # a: 예측값을 담을 데이터프레임 생성

        for i in range(24):
            a[str(i+1)]=[fcst1[0][i]] # column명 지정 및 예측값 대입

        # a 에 meter_id를 현재 예측하고 있는 열의 id(key)로 대입
        a['meter_id'] = key 

        # agg{ 미터ID: 시간별(subimssion.columns는 예측시간칼럼들) a의 예측값}
        agg[key] = a[hrs]
    print('---- Modeling Done ----')
    return agg

In [31]:
aggTEST = runArima(ch_te_TimeCut)

1 X26 predicting
2 X303 predicting
3 X241 predicting
4 X435 predicting
5 X402 predicting
6 X352 predicting
7 X305 predicting
8 X350 predicting
9 X326 predicting
10 X299 predicting
11 X160 predicting
12 X125 predicting
13 X211 predicting
14 X349 predicting
15 X407 predicting
16 X330 predicting
17 X132 predicting
18 X231 predicting
19 X225 predicting
20 X16 predicting
21 X286 predicting
22 X354 predicting
23 X353 predicting
24 X76 predicting
25 X255 predicting
26 X106 predicting
27 X393 predicting
28 X157 predicting
29 X323 predicting
30 X130 predicting
31 X269 predicting
32 X94 predicting
33 X172 predicting
34 X289 predicting
35 X216 predicting
36 X123 predicting
37 X398 predicting
38 X233 predicting
39 X214 predicting
40 X270 predicting
41 X371 predicting
42 X200 predicting
43 X111 predicting
44 X163 predicting
45 X364 predicting
46 X378 predicting
47 X271 predicting
48 X462 predicting
49 X427 predicting
50 X73 predicting
51 X7 predicting
52 X263 predicting
53 X450 predicting
54 X18 pr

In [35]:
output1 = pd.concat(aggTEST, ignore_index=False)
output2 = output1.T
output2.columns = output2.columns.droplevel(level = 1)
hrsTest_pred = output2

In [39]:
hrsTest_pred.to_csv("data/TEST_DHWonDacon_24시간예측.csv")
# from IPython.display import FileLink
# FileLink(r"TRAIN_DHWonDacon_24시간192시간예측.csv")

In [40]:
aggTrain = runArima(ch_tr_TimeCut)

1 X692 predicting
2 X1272 predicting
3 X553 predicting
4 X1299 predicting
5 X598 predicting
6 X1003 predicting
7 X1010 predicting
8 X1216 predicting
9 X1047 predicting
10 X381 predicting
11 X466 predicting
12 X1420 predicting
13 X686 predicting
14 X711 predicting
15 X1251 predicting
16 X828 predicting
17 X1350 predicting
18 X1066 predicting
19 X1293 predicting
20 X1148 predicting
21 X451 predicting
22 X174 predicting
23 X74 predicting
24 X626 predicting
25 X1083 predicting
26 X1000 predicting
27 X632 predicting
28 X1380 predicting
29 X607 predicting
30 X680 predicting
31 X977 predicting
32 X492 predicting
33 X968 predicting
34 X891 predicting
35 X122 predicting
36 X489 predicting
37 X86 predicting
38 X392 predicting
39 X1007 predicting
40 X549 predicting
41 X1343 predicting
42 X437 predicting
43 X551 predicting
44 X287 predicting
45 X138 predicting
46 X339 predicting
47 X1210 predicting
48 X127 predicting
49 X644 predicting
50 X1472 predicting
51 X604 predicting
52 X1356 predicting
53 

410 X415 predicting
411 X198 predicting
412 X805 predicting
413 X360 predicting
414 X526 predicting
415 X1154 predicting
416 X1153 predicting
417 X956 predicting
418 X702 predicting
419 X590 predicting
420 X1151 predicting
421 X401 predicting
422 X36 predicting
423 X1239 predicting
424 X454 predicting
425 X594 predicting
426 X681 predicting
427 X850 predicting
428 X153 predicting
429 X1355 predicting
430 X602 predicting
431 X781 predicting
432 X1479 predicting
433 X1262 predicting
434 X903 predicting
435 X874 predicting
436 X1254 predicting
437 X273 predicting
438 X318 predicting
439 X748 predicting
440 X599 predicting
441 X636 predicting
442 X837 predicting
443 X1237 predicting
444 X1032 predicting
445 X1208 predicting
446 X1101 predicting
447 X751 predicting
448 X1411 predicting
449 X946 predicting
450 X1443 predicting
451 X1480 predicting
452 X1488 predicting
453 X1036 predicting
454 X821 predicting
455 X332 predicting
456 X501 predicting
457 X1245 predicting
458 X760 predicting
459

814 X1432 predicting
815 X463 predicting
816 X1325 predicting
817 X1105 predicting
818 X1055 predicting
819 X1354 predicting
820 X1058 predicting
821 X1449 predicting
822 X147 predicting
823 X753 predicting
824 X583 predicting
825 X1001 predicting
826 X97 predicting
827 X1489 predicting
828 X806 predicting
829 X1310 predicting
830 X881 predicting
831 X1273 predicting
832 X1398 predicting
833 X809 predicting
834 X861 predicting
835 X1461 predicting
836 X907 predicting
837 X188 predicting
838 X1315 predicting
839 X394 predicting
840 X1494 predicting
841 X745 predicting
842 X1122 predicting
843 X497 predicting
844 X847 predicting
845 X817 predicting
846 X1300 predicting
847 X264 predicting
848 X395 predicting
849 X584 predicting
850 X1205 predicting
851 X645 predicting
852 X322 predicting
853 X335 predicting
854 X1385 predicting
855 X1092 predicting
856 X1016 predicting
857 X1452 predicting
858 X1460 predicting
859 X193 predicting
860 X624 predicting
861 X1030 predicting
862 X1424 predict

1207 X11 predicting
1208 X190 predicting
1209 X1063 predicting
1210 X25 predicting
1211 X234 predicting
1212 X222 predicting
1213 X1223 predicting
1214 X1277 predicting
1215 X980 predicting
1216 X550 predicting
1217 X661 predicting
1218 X84 predicting
1219 X1176 predicting
1220 X858 predicting
1221 X596 predicting
1222 X1265 predicting
1223 X1391 predicting
1224 X715 predicting
1225 X1419 predicting
1226 X682 predicting
1227 X444 predicting
1228 X1192 predicting
1229 X328 predicting
1230 X171 predicting
1231 X570 predicting
1232 X720 predicting
1233 X505 predicting
1234 X852 predicting
1235 X1279 predicting
1236 X1225 predicting
1237 X802 predicting
1238 X499 predicting
1239 X440 predicting
1240 X1166 predicting
1241 X1431 predicting
1242 X1482 predicting
1243 X23 predicting
1244 X441 predicting
1245 X1070 predicting
1246 X707 predicting
1247 X493 predicting
1248 X785 predicting
1249 X873 predicting
1250 X1179 predicting
1251 X1125 predicting
1252 X109 predicting
1253 X224 predicting
1

In [41]:
output1 = pd.concat(aggTrain, ignore_index=False)
output2 = output1.T
output2.columns = output2.columns.droplevel(level = 1)
hrsTrain_pred = output2

In [46]:
hrsTrain_pred.to_csv("data/TRAIN_DHWonDacon_24시간예측.csv")
# from IPython.display import FileLink
# FileLink(r"TSET_DHWonDacon_24시간192시간예측.csv")